In [1]:
import pandas as pd

# Define File Paths & Load to DataFrame

In [3]:
BASE_PATH = './2025_A2/train'
METADATA_PATH = BASE_PATH + '/train_metadata.csv'
metadata = pd.read_csv(METADATA_PATH)   # columns: image_id, ClassId
# test_meta  = pd.read_csv("test_metadata.csv")    # columns: image_id

FEATURE_PATH = BASE_PATH + '/Features'

print(FEATURE_PATH + "/additional_features.csv")
add_feats  = pd.read_csv(FEATURE_PATH + "/additional_features.csv")  # edge_density, mean_b/g/r …
color_hist = pd.read_csv(FEATURE_PATH + "/color_histogram.csv")      # 95 colour-histogram bins
hog_pca    = pd.read_csv(FEATURE_PATH + "/hog_pca.csv")           # 100 PCA components

./2025_A2/train/Features/additional_features.csv


# Data Processing

### Join evety feature CSV with ClassID in metadata

In [ ]:
hog_pca = hog_pca.merge(
    metadata[["image_path", "ClassId"]],
    on="image_path",
    how="left"
)



Index(['image_path', 'hog_pca_0', 'hog_pca_1', 'hog_pca_2', 'hog_pca_3',
       'hog_pca_4', 'hog_pca_5', 'hog_pca_6', 'hog_pca_7', 'hog_pca_8',
       'hog_pca_9', 'hog_pca_10', 'hog_pca_11', 'hog_pca_12', 'hog_pca_13',
       'hog_pca_14', 'hog_pca_15', 'hog_pca_16', 'hog_pca_17', 'hog_pca_18',
       'hog_pca_19', 'ClassId'],
      dtype='object')
